In [13]:
!pip install --upgrade pip

In [14]:
!pip install aitextgen
!pip install tensorflow

In [3]:
!pip install ipywidgets

In [4]:
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu116


In [5]:
import json

# Open the dataset JSON
with open("dataSets/ingredient_and_instructions.json", "r") as dataSet:
    data = json.load(dataSet)

recipie = "" # The individual recipie
recipie_list = [] # Where we will store the recipie for training

for meal in data:
    # Get the recipie name and its instructions
    meal_info = data[meal]
    recipie += meal.replace("-", " ").title() + "\n\n"
    recipe_instructions = meal_info["instructions"]

    # Get the ingredients
    ingredients = meal_info["ingredient_sections"]

    for i in ingredients:
        # Retrieve ingredients and their details
        item = ""           # Dough
        qty = ""            # ½
        unit = ""           # cup
        ingredient = ""     # unsalted butter
        primary_unit = { "quantity": None, "display": None }

        # Some ingredients arent named, but are still listed as "1 cup" or similar
        # Additionally, if there is an ingredient name, it may already have a colon
        # in front of it, so we need to check for that before formatting
        if i["name"]:
            if i["name"][-1] != ":": # If there isnt a colon, add one
                item = i["name"] + ": \n"
            else: # Otherwise, just add the name
                item = i["name"] + " \n"
        else:
            item = ""

        recipie += item # Add the ingredient name to the recipie

        # Loop over each ingredient
        j = 0
        while j < len(i["ingredients"]):
            # i["ingredients"][0] may not exist, use try/except to prevent
            # any ListIndexErrors from accessing an index that doesn't exist
            try: primary_unit = i["ingredients"][j]["primary_unit"]
            except: primary_unit = { "quantity": None, "display": None } # Dummy object

            # Some recipies don't list an ingredient at all
            try: ingredient += i["ingredients"][j]["name"]
            except: ingredient += ""

            # If the try/except blocks above are successful, we can set the values
            if primary_unit["quantity"]: qty = primary_unit["quantity"] + " "
            if primary_unit["display"]: unit = primary_unit["display"] + " "

            # Construct the ingredient string
            recipie += qty + unit  + ingredient + "\n"
            ingredient = ""
            j += 1

        recipie += "\n"

    # Newline before adding instructions
    recipie += "\n"

    # Loop over the instructions and format them
    step = 1
    for text in recipe_instructions:
        instruction = text["display_text"]
        recipie += str(step) + ". " + instruction + "\n"
        step += 1

    # Add the recipie to the list
    recipie_list.append(recipie)

    # Free memory for the next recipie
    recipie = ""
    recipe_instructions = []
    ingredients = []
    item = ""
    qty = ""
    unit = ""
    ingredient = ""

# For testing purposes, we can output a random recipie
import random
print(recipie_list[random.randint(0, len(recipie_list) - 1)])

#print(recipie_list[0])


Creamy Pesto Pasta Bake

Pesto: 
4 cups fresh basil
2 cloves garlic
½ cup pine nuts
¾ cup grated parmesan cheese
¾ cup olive oil
¼ cup cold water
½ cup heavy cream
1 teaspoon salt
½ teaspoon pepper

Pasta Bake: 
12 oz fusilli pasta
1 cup cherry tomato
1 cup grated parmesan cheese
8 oz mozzarella ball


1. To make the pesto, mix basil, garlic, pine nuts, parmesan, cold water, heavy cream, salt, and pepper in a food processor for about 30 seconds. Add in olive oil. Process until combined, set aside.
2. Cook fusilli to very al dente, or about 3-4 minutes before the package instructs for cook time, then drain pasta.
3. Halve cherry tomatoes.
4. In a skillet or casserole pan, mix fusilli, cherry tomatoes, and pesto.
5. Top with grated parmesan and mozzarella balls.
6. Bake at 400° Fahrenheit for 10-15 minutes, or until mozzarella has melted.
7. Garnish with additional parmesan and fresh basil. Serve immediately.
8. Enjoy!



In [6]:
#CSV File 1 Record per row
#Record: 1 string with ingredients + recipe


# [[recipe1], [recipe2], [recipe3]]

import csv

with open("recipe_fixed.csv", 'w', encoding="utf-8") as csvfile:
    csvwriter = csv.writer(csvfile)
    for recipie in recipie_list:
        csvwriter.writerow([recipie])


--- MODEL TRANING ---

In [5]:

!python3 -m pip install tensorflow
# Verify install:
!python3 -c "import tensorflow as tf; print(tf.config.list_physical_devices('GPU'))"

[]


2022-07-29 20:32:21.122808: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-07-29 20:32:21.122972: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-07-29 20:32:22.883870: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-07-29 20:32:22.884068: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cublas64_11.dll'; dlerror: cublas64_11.dll not found
2022-07-29 20:32:22.884271: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cublasLt64_11.dll'; dlerror: cublasLt64_11.dll not found
2022-07-29 20:32:22.884505: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cu

In [1]:
#check if cuda is installed
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))
#Check tesnsorflow version
print(tf.__version__)
#Check tensorflow GPU version
print(tf.test.gpu_device_name())


[]
2.9.1



In [2]:
from aitextgen.TokenDataset import TokenDataset
from aitextgen.tokenizers import train_tokenizer
from aitextgen.utils import GPT2ConfigCPU
from aitextgen import aitextgen
import numpy

tokenizer_file = "aitextgen.tokenizer.json"
config = GPT2ConfigCPU()
ai = aitextgen(tokenizer_file=tokenizer_file, tf_gpt2="774M", to_gpu=True)

AssertionError: CUDA is not installed.

In [ ]:
#Ai training disabled to prevent accidental overrides.

file_name = "recipe_fixed.csv"
train_tokenizer(file_name)
data = TokenDataset(file_name, tokenizer_file=tokenizer_file, block_size=64)
ai.train(data, batch_size=40, num_steps=64000, generate_every=500, save_every=2000)
ai.generate(prompt="cherry pie")

You are tokenizing a CSV file, but you did not set line_by_line=True. Please change if unintended.
100%|██████████| 9922/9922 [00:00<00:00, 14021.79it/s]
Windows does not support multi-GPU training. Setting to 1 GPU.
C:\Users\flash\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pytorch_lightning\trainer\connectors\callback_connector.py:151: LightningDeprecationWarning: Setting `Trainer(checkpoint_callback=False)` is deprecated in v1.5 and will be removed in v1.7. Please consider using `Trainer(enable_checkpointing=False)`.
  rank_zero_deprecation(
C:\Users\flash\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pytorch_lightning\trainer\connectors\callback_connector.py:96: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=20)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.pr

  0%|          | 0/64000 [00:00<?, ?it/s]

C:\Users\flash\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pytorch_lightning\trainer\trainer.py:726: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x000001C40063F310>
Traceback (most recent call last):
  File "C:\Users\flash\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\torch\utils\data\dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "C:\Users\flash\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\torch\utils\data\dataloader.py", line 1295, in _shutdown_workers
    if self._persistent_workers or self._workers_status[worker_id]:
AttributeError: '_MultiProcessin

cherry pie�ccalth into� su teaspoonservingpperpper edges beat car� Preransferwocol cloves� honeyY� 350 stickillzz potatoes p Add batter oliveY warm3ol cheeseine� warm b combined cmtss� more�ransfer warm com� 350otherw


In [ ]:
# Load the trained model
# ai = aitextgen(model_folder="trained_model", tokenizer_file="aitextgen.tokenizer.json")
# ai.generate(temperature=.4 , prompt="steak", max_length=100)

steaks.
5. Serve with avocado crema and fresh cilantro.
6. Enjoy!
One Pot Shrimp Lo Mein

8 oz little more oil
2 tablespoons. Add the chicken breasts, season with salt and pepper, and fry powder
3 cloves garlic
1 tablespoon soy sauce



Caji
